In [1]:
import numpy as np
import scipy.integrate as integrate
from scipy.signal import fftconvolve
import os,sys
import yaml
import matplotlib.pyplot as plt
import math
import cv2

In [2]:
def sample_coordinate(pixelsize_x = 55e-06,pixelsize_y = 55e-06,fs_size = 2000,ss_size = 2000,focus_x = 1.2e-03,focus_y = 1.0e-03,defocus = 400e-06, det_dist = 14.0e-03, ap_x = 40e-06, ap_y= 40e-06,wl = 7.29e-11):

    xx_span = fs_size * pixelsize_x
    yy_span = ss_size * pixelsize_y 
    x_span_obj =  ap_x / focus_x * defocus 
    y_span_obj =  ap_y / focus_y * defocus
    print(x_span_obj,y_span_obj)
    sintheta_x = (ap_x/2)/np.sqrt((ap_x/2)**2+focus_x**2)
    sintheta_y = (ap_y/2)/np.sqrt((ap_y/2)**2+focus_y**2)
    print(sintheta_x,sintheta_y)
    n_x = 2048
    #n_x = int(np.maximum(x_span_obj,ap_x) / wl * 2* sintheta_x)
    n_y = 2048
    #n_y = int(np.maximum(y_span_obj,ap_y) / wl * 2* sintheta_y)

    print(n_x,n_y)

    delta_xlen = ap_x / n_x
    delta_ylen = ap_y / n_y
    delta_xobj = x_span_obj/n_x
    delta_yobj = y_span_obj/n_y
    delta_xdet = xx_span/n_x
    delta_ydet = yy_span/n_y
    
    return delta_xlen,delta_ylen,delta_xobj,delta_yobj,delta_xdet,delta_ydet,n_x,n_y

In [3]:
with open('configure_simtry.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)

In [4]:
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

ss_size = conf_para['Detector']['ss_size']
fs_size = conf_para['Detector']['fs_size']

pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
wl = conf_para['Source']['wl']
#wl = 7.29e-10
k = 2*np.pi/wl

In [5]:
delta_xlen,delta_ylen,delta_xobj,delta_yobj,delta_xdet,delta_ydet,n_x,n_y = sample_coordinate(pixelsize_x = pixelsize_x,pixelsize_y = pixelsize_y,fs_size = fs_size,ss_size = ss_size,focus_x = focus_x,focus_y = focus_y,defocus = defocus, det_dist = det_dist, ap_x = ap_x, ap_y= ap_y,wl = wl)

0.00013333333333333337 0.00016
0.016664352333993337 0.01999600119960014
2048 2048


In [6]:
p_x = np.arange(-n_x//2,n_x//2)
p_xa = p_x * delta_xdet
p_xx = p_xa[:,np.newaxis]
p_y = np.arange(-n_y//2,n_y//2)
p_ya = p_y * delta_ydet
p_yy = p_ya[np.newaxis,:]

In [7]:
pupil_focus = np.zeros((p_y.size,p_x.size))
filter_illuminated_indices_atf = np.where( (np.abs(p_xx) < (ap_x/2)) & (np.abs(p_yy) < (ap_y/2)))
#p_atf[filter_illuminated_indices_atf] = np.exp(-1.0j*k/zi*(filter_illuminated_indices_atf[0]*delta_xdet+ filter_illuminated_indices_atf[1]*delta_ydet))
pupil_focus[filter_illuminated_indices_atf] = 1
print(filter_illuminated_indices_atf)
#p_atf = np.exp(-1.0j*k/zi*(xx_arr+yy_arr))

(array([ 950,  950,  950, ..., 1098, 1098, 1098]), array([ 950,  951,  952, ..., 1096, 1097, 1098]))


In [8]:
%matplotlib widget
x_min = np.min(p_xx)
x_max = np.max(p_xx)
y_min = np.min(p_yy)
y_max = np.max(p_yy)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p1 = axes[0].imshow(np.abs(pupil_focus),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude of input pupil', fontsize=14)
p2 = axes[1].imshow(np.unwrap(np.angle(pupil_focus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase of input pupil', fontsize=14)
plt.colorbar(p1,ax = axes[0])
plt.colorbar(p2,ax = axes[1])
plt.savefig('Input_pupilatf_210107.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
npixels_x = p_x.size
freq_nyquistx = 0.5/pixelsize_x
freq_n = np.linspace(-1.0,1.0,npixels_x)
freq_nn = np.fft.fftfreq(npixels_x)
freq_x = freq_n*freq_nyquistx

npixels_y = p_y.size
freq_nyquisty = 0.5/pixelsize_y
freq_n = np.linspace(-1.0,1.0,npixels_y)
freq_y = freq_n*freq_nyquisty

freq_xy = np.array(np.meshgrid(freq_y,freq_x))
print(freq_n.size)

2048


In [119]:
pxy = np.array(np.meshgrid(p_ya,p_xa))
def W(x,y,focus_x,focus_y,alpha_x,alpha_y,wl):
    w = np.pi/wl*(x**2/focus_x+y**2/focus_y)+1e9*(alpha_x*(x/focus_x)**3+alpha_y*(y/focus_y)**3)
    return w

In [127]:
fx_arr = np.fft.fftfreq(n_x)
fy_arr = np.fft.fftfreq(n_y)
f_xy_arr = np.array(np.meshgrid(fy_arr,fx_arr))
h2_atf = np.fft.fftshift(np.fft.fft2(pupil_focus))*np.exp(1.0j*W(freq_xy[1]*wl*focus_x,freq_xy[0]*wl*focus_y,focus_x,focus_y,alpha_x,alpha_y,wl))
p_focus_out = np.fft.ifft2(h2_atf)

In [138]:
%matplotlib widget
x_min = np.min(p_xx)
x_max = np.max(p_xx)
y_min = np.min(p_yy)
y_max = np.max(p_yy)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p3 = axes[0].imshow(np.abs(p_focus_out),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude of ATF', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(p_focus_out)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase of ATF', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('ATF_210108.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
%matplotlib widget
plt.plot(np.unwrap(np.angle(p_focus_out[1024])))
#plt.plot(np.abs(p_out[2000]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Propagate from focus → object plane

In [141]:
fx_arr = np.fft.fftfreq(n_x)
fy_arr = np.fft.fftfreq(n_y)
f_xy_arr = np.array(np.meshgrid(fy_arr,fx_arr))
z = defocus
h2_obj = h2_atf*np.exp(-1.j*np.pi*wl*z*(np.fft.fftshift(freq_xy[0]**2 + freq_xy[1]**2)))
p_defocus = np.fft.ifft2(h2_obj)

In [142]:
%matplotlib widget
x_min = np.min(p_xx)
x_max = np.max(p_xx)
y_min = np.min(p_yy)
y_max = np.max(p_yy)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p3 = axes[0].imshow(np.abs(p_defocus),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude of ATF', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(p_defocus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase of ATF', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('ATF_210107.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
%matplotlib widget
plt.plot(np.unwrap(np.angle(p_defocus[1024])))
#plt.plot(np.abs(p_out[2000]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 1D Try

In [329]:
x_arr = np.arange(-pupil_input.shape[1]//2,pupil_input.shape[1]//2)*delta_xdet
fx_arr = wl*zi*2*np.pi/n_x*np.fft.fftfreq(pupil_input.shape[1])
p_atf = np.zeros((n_x))
filter_illuminated_indices_x = np.where( (np.abs(x_arr) < (ap_x*defocus/focus_x/2)))
p_atf[filter_illuminated_indices_x] = 1
za = focus_x
zi = det_dist
def Wm1d(za,zi,width,alpha_x):
    wm = -0.5*(1/za-1/zi)*width**2
    return wm

def W1d(x,za,zi,width,focus_x,alpha_x):
    w = np.exp(1.0j*k*Wm1d(za,zi,width,alpha_x)*(x**2)/width**2+1e9j*alpha_x*(x/focus_x)**3)
    return w

h_atf = np.fft.fftshift(np.fft.fft(p_atf)*W1d(fx_arr,za,zi,width,focus_x,alpha_x))

In [330]:
p_in = np.zeros(x_arr.shape)
p_in[np.abs(x_arr)< (ap_x/2)]=1
p_out = np.fft.ifftshift(np.fft.ifft(np.fft.fft(p_in)*h_atf))

In [331]:
%matplotlib widget
#plt.plot(np.unwrap(np.angle(p_output[2000])))
plt.plot(np.abs(p_out))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [332]:
%matplotlib widget
plt.plot(x_arr,np.unwrap(np.angle(p_out)))
#plt.plot(x_arr,np.abs(p_out))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …